In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
from keras import layers
import os
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler
from tabulate import tabulate
from itertools import islice
from scipy.signal import resample
from sklearn.preprocessing import StandardScaler
import pickle
import glob
import random

In [129]:
data_path = '/content/drive/MyDrive/MovePort/'
#Modified_Df_PATH = '/content/drive/MyDrive/MovePortModif/'
participants = sorted([p for p in os.listdir(data_path) if p.isdigit()], key=int)
print("Participants disponibles :", participants)
activities = ['back', 'forward', 'halfsquat', 'still']
modalities = ['ips_1', 'emg_1', 'imu_1']

Participants disponibles : ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25']


In [130]:
def list_missing_files(base_path, participants, activities, modalities):
    """
        base_path (str): Le chemin de base contenant les dossiers de participants.
        participants (list): Liste des IDs des participants.
        activities (list): Liste des activités (back, forward, halfsquat, still).
        modalities (list): Liste des modalités (ips, emg, imu).
    """
    missing_files = {}

    for participant in participants:
        participant_missing = {}
        for activity in activities:
            for modality in modalities:
                file_path = os.path.join(base_path, str(participant), activity, f"{modality}.csv")
                if not os.path.exists(file_path):
                    if activity not in participant_missing:
                        participant_missing[activity] = []
                    participant_missing[activity].append(modality)
        if participant_missing:
            missing_files[participant] = participant_missing

    return missing_files

In [131]:
# Lister les fichiers manquants
missing_files = list_missing_files(data_path, participants, activities, modalities)
# Afficher les résultats
participants_manque_files = []
print("Participants avec des fichiers de modalités manquants :")
for participant, activities in missing_files.items():
    print(f"Participant {participant}:")
    participants_manque_files.append(participant)
    for activity, modalities in activities.items():
        print(f"  Activité {activity}: Modalités manquantes: {', '.join(modalities)}")

Participants avec des fichiers de modalités manquants :
Participant 22:
  Activité back: Modalités manquantes: ips_1, emg_1, imu_1


In [123]:
participants_manque_files

['18', '19', '20', '21', '22', '23']

In [132]:
def get_column_structure_and_shape(base_path, participants, activities, modalities):
    """
    Récupère la structure des colonnes et la forme des données pour chaque modalité.
    Args:
        base_path (str): Le chemin de base contenant les dossiers de participants.
        participants (list): Liste des IDs des participants existants.
        activities (list): Liste des activités (back, forward, halfsquat, still).
        modalities (list): Liste des modalités (ips_1, emg_1, imu_1).
    Returns:
        dict: Un dictionnaire contenant les structures de colonnes et les formes pour chaque modalité.
    """
    column_structure = {}
    data_shapes = {}
    for modality in modalities:
        for participant in participants:
            for activity in activities:
                file_path = os.path.join(base_path, str(participant), activity, f"{modality}.csv")
                if os.path.exists(file_path):
                    df = pd.read_csv(file_path, header=0)
                    column_structure[modality] = df.columns
                    data_shapes[modality] = df.shape
                    break
            if modality in column_structure:
                break
        if modality not in column_structure:
            raise ValueError(f"Structure des colonnes introuvable pour la modalité {modality}")

    return column_structure, data_shapes

In [133]:
def calculate_mean_structure(base_path, participants, activities, modalities):

    mean_data = {modality: [] for modality in modalities}

    for activity in activities:
        for modality in modalities:
            participant_pairs = random.sample(participants, 2)
            modality_data = []
            for participant in participant_pairs:
                file_path = os.path.join(base_path, str(participant), activity, f"{modality}.csv")
                if os.path.exists(file_path):
                    try:
                        df = pd.read_csv(file_path, header=0)
                        numeric_df = df.apply(pd.to_numeric, errors='coerce')  # Convertir en numérique
                        modality_data.append(numeric_df)
                    except Exception as e:
                        print(f"Erreur lors du chargement de {file_path}: {e}")
            if modality_data:
                combined_df = pd.concat(modality_data)
                combined_df = combined_df.apply(pd.to_numeric, errors='coerce')
                combined_df = combined_df.dropna(axis=1, how='all')  # Supprimer les colonnes entièrement non numériques
                mean_df = combined_df.mean(axis=0).to_frame().transpose()  # Calculer uniquement les colonnes numériques
                mean_df = round(mean_df, 3)
                mean_data[modality].append(mean_df)

    for modality in modalities:
        mean_data[modality] = pd.concat(mean_data[modality], axis=0).mean(axis=0).to_frame().transpose()

    return mean_data

In [134]:
def create_missing_files_with_mean(base_path, participant_id, activities, modalities, mean_data, column_structure, data_shapes):

    participant_path = os.path.join(base_path, str(participant_id))
    for activity in activities:
        activity_path = os.path.join(participant_path, activity)
        if not os.path.exists(activity_path):
            os.makedirs(activity_path)  # Crée le dossier d'activité s'il n'existe pas
        for modality in modalities:
            file_path = os.path.join(activity_path, f"{modality}.csv")
            if not os.path.exists(file_path):
                print(f"Création de {file_path} avec des données moyennes et structure correcte")
                shape = data_shapes[modality]
                synthetic_data = pd.DataFrame(np.tile(mean_data[modality].values, (shape[0], 1)), columns=mean_data[modality].columns)
                synthetic_data = synthetic_data.reindex(columns=column_structure[modality], fill_value=0)  # Alignement des colonnes
                synthetic_data.to_csv(file_path, index=False)

In [135]:
# Calculer les moyennes à partir des participants 1 à 16
existing_participants = list(range(1, 17))

In [136]:
# Créer les fichiers manquants pour chaque participant de la liste
column_structure, data_shapes = get_column_structure_and_shape(data_path, existing_participants, activities, modalities)
for participant_id in participants_manque_files:
    mean_data = calculate_mean_structure(data_path, existing_participants, activities, modalities)
    create_missing_files_with_mean(data_path, participant_id, activities, modalities, mean_data, column_structure, data_shapes)

print("Tous les dossiers et fichiers manquants ont été créés avec des valeurs moyennes et une structure correcte.")

Création de /content/drive/MyDrive/MovePort/22/back/ips_1.csv avec des données moyennes et structure correcte
Création de /content/drive/MyDrive/MovePort/22/back/emg_1.csv avec des données moyennes et structure correcte
Création de /content/drive/MyDrive/MovePort/22/back/imu_1.csv avec des données moyennes et structure correcte
Tous les dossiers et fichiers manquants ont été créés avec des valeurs moyennes et une structure correcte.


In [138]:
newFileips = pd.read_csv('/content/drive/MyDrive/MovePort/18/forward/ips_1.csv')
newFileemg = pd.read_csv('/content/drive/MyDrive/MovePort/19/forward/emg_1.csv')
newFileimu = pd.read_csv('/content/drive/MyDrive/MovePort/21/forward/imu_1.csv')
print('shape of ips' ,newFileips.shape)
print('shape of emg' ,newFileemg.shape)
print('shape of imu' ,newFileimu.shape)

shape of ips (682, 3605)
shape of emg (16, 120161)
shape of imu (54, 6009)


In [139]:
# Fonction pour inverser le CSV (transposer les données)
def inverser_csv(file_path):
    """Lire un fichier CSV et le transposer."""
    df = pd.read_csv(file_path, header=0)
    df = df.transpose()
    df.columns = df.iloc[0]  # Première ligne devient l'en-tête
    df = df[1:].reset_index(drop=True)  # Supprimer la première ligne et réindexer
    return df

In [ ]:
def process_data_in_batch(participant_path, modality):
    """Traitement des fichiers dans un batch pour une modalité spécifique."""

    # Catégories intégrées directement dans la fonction
    categories = ['back', 'forward', 'halfsquat', 'still']

    # Dictionnaire pour mapper les catégories aux valeurs numériques
    category_mapping = {'back': 1, 'forward': 2, 'halfsquat': 3, 'still': 4}

    all_category_data = []  # Liste pour stocker les données des différentes catégories pour un participant

    for category in categories:
        # Chercher les fichiers spécifiques à la modalité
        files = glob.glob(os.path.join(participant_path, category, f"{modality}_*.csv"))
        print(f"  Fichiers {modality.upper()} trouvés pour la catégorie '{category}': {files}")

        category_data = []  # Liste pour stocker les données de cette catégorie

        for file in files:
            print(f"    Traitement du fichier {modality.upper()} : {file}")
            try:
                df = inverser_csv(file)  # Inverser les données (transposer)
                df['categorie'] = category_mapping[category]  # Ajouter la catégorie encodée
                category_data.append(df)  # Ajouter au lot de la catégorie
            except Exception as e:
                print(f"Erreur lors du traitement du fichier {file}: {e}")

        # Concaténer toutes les données de la catégorie pour ce participant
        if category_data:
            concatenated_category_data = pd.concat(category_data, ignore_index=True)
            all_category_data.append(concatenated_category_data)

    # Retourner toutes les données des catégories pour ce participant
    if all_category_data:
        return pd.concat(all_category_data, ignore_index=True)
    else:
        return None

In [ ]:

def process_participants(data_path, participants, modality, batch_size=5):
    """
    Traite les fichiers de plusieurs participants pour une modalité spécifique (EMG, IMU ou IPS).
    Ajoute des colonnes 'id' et 'categorie' et retourne les données concaténées.
    """
    all_data = []  # Liste pour stocker toutes les données traitées

    # Diviser les participants en lots (batchs)
    participants_batches = [participants[i:i + batch_size] for i in range(0, len(participants), batch_size)]

    for batch_idx, batch_participants in enumerate(participants_batches, start=1):
        print(f"Traitement du lot {batch_idx}/{len(participants_batches)} : Participants {batch_participants}")
        batch_data = []  # Liste pour stocker les données d'un batch

        for participant_id in batch_participants:
            participant_path = os.path.join(data_path, str(participant_id))

            # Traiter les données pour ce participant
            print(f"Participant {participant_id}")
            participant_data = process_data_in_batch(participant_path, modality)

            # Ajouter l'ID du participant aux données
            if participant_data is not None:
                participant_data['id'] = participant_id  # Ajouter l'ID du participant
                batch_data.append(participant_data)

        # Concaténer les données du lot
        if batch_data:
            batch_df = pd.concat(batch_data, ignore_index=True)
            all_data.append(batch_df)

    # Concaténer toutes les données traitées en un seul DataFrame
    if all_data:
        concatenated_data = pd.concat(all_data, ignore_index=True)
        return concatenated_data
    else:
        print("Aucune donnée valide trouvée.")
        return None

In [ ]:
# Exemple d'appel à la fonction
data_path = '/content/drive/MyDrive/MovePort'  # Chemin vers les données
participants = [1,2]  # Liste des participants
modality = 'ips'  # Modalité, ici EMG
batch_size = 5  # Taille du lot

# Appel de la fonction pour traiter les données
processed_data = process_participants(data_path, participants, modality, batch_size)

# Vérifier les premières lignes du DataFrame traité
processed_data.head(10)

In [ ]:
def concatener_emg_files(data_folder):
    all_data = []

    # Parcourir les dossiers numérotés de 1 à 25
    for person_id in range(1, 26):
        person_folder = os.path.join(data_folder, str(person_id))

        # Parcourir chaque catégorie (back, forward, halfsquat, still)
        for category in ['back', 'forward', 'halfsquat', 'still']:
            category_folder = os.path.join(person_folder, category)

            # Chercher tous les fichiers emg_*.csv dans le dossier de la catégorie
            emg_files = glob.glob(os.path.join(category_folder, "emg_*.csv"))

            for file in emg_files:
                # Inverser le fichier CSV
                df = inverser_csv(file)

                # Ajouter les colonnes id et categorie
                df['id'] = person_id
                df['categorie'] = category

                # Ajouter à la liste des DataFrames
                all_data.append(df)

    # Concaténer tous les DataFrames en un seul
    if all_data:
        concatenated_df = pd.concat(all_data, ignore_index=True)

        # Enregistrer le fichier final
        concatenated_df.to_csv('emg_concatenee2.csv', index=False)
        print("Fichier 'hameed.csv' créé avec succès.")
    else:
        print("Aucun fichier EMG trouvé.")

#concatener_emg_files(data_path)

In [ ]:
def process_participants_by_modality(data_path, participants, categories, modality, output_file="processed_data.csv"):
    """
    Traite les fichiers de plusieurs participants pour une modalité spécifique (EMG, IMU ou IPS).
    Ajoute des colonnes 'id' et 'categorie' et renvoie les données concaténées.
    """
    all_data = []  # Pour stocker toutes les données concaténées

    # Parcourir les participants
    for participant_id in participants:
        participant_path = os.path.join(data_path, str(participant_id))

        for category in categories:
            category_path = os.path.join(participant_path, category)

            # Chercher les fichiers spécifiques à la modalité
            files = glob.glob(os.path.join(category_path, f"{modality}_*.csv"))

            print(f"Participant {participant_id}, Catégorie : {category}")
            print(f"  Fichiers {modality.upper()} : {files}")

            category_data = []  # Liste temporaire pour stocker les données de la catégorie actuelle

            # Traiter chaque fichier
            for file in files:
                print(f"    Traitement du fichier {modality.upper()} : {file}")
                try:
                    df = inverser_csv(file)  # Utiliser la fonction d'inversion
                    df['id'] = participant_id  # Ajouter l'ID du participant
                    df['categorie'] = category  # Ajouter la catégorie
                    category_data.append(df)  # Ajouter au lot de la catégorie
                except Exception as e:
                    print(f"Erreur lors du traitement du fichier {file}: {e}")

            # Ajouter les données de la catégorie au lot global
            if category_data:
                concatenated_category_data = pd.concat(category_data, axis=0, ignore_index=True)
                all_data.append(concatenated_category_data)

    # Concaténer toutes les données en un seul DataFrame
    if all_data:
        concatenated_data = pd.concat(all_data, axis=0, ignore_index=True)

        # Sauvegarder dans un fichier CSV
        concatenated_data.to_csv(output_file, index=False)
        print(f"Les données ont été sauvegardées dans le fichier {output_file}.")

        return concatenated_data
    else:
        print("Aucune donnée valide trouvée.")
        return None

In [ ]:
# Nettoyage des données
def clean_missing_values(processed_data, threshold=0.51):
    cleaned_data = {}

    for participant, activities in processed_data.items():
        cleaned_data[participant] = {}
        for activity, modalities in activities.items():
            cleaned_data[participant][activity] = {}
            for modality, df in modalities.items():
                # Étape 1 : Supprimer les lignes contenant >= threshold% de zéros
                zero_threshold = (df == 0).mean(axis=1)  # Calcul du pourcentage de zéros par ligne
                df = df.loc[zero_threshold < threshold]  # Conserver les lignes où le pourcentage de zéros est inférieur au seuil
                # Étape 2 : Identifier et traiter uniquement les colonnes numériques
                numeric_cols = df.select_dtypes(include=[np.number])
                non_numeric_cols = df.select_dtypes(exclude=[np.number])
                # Étape 3 : Calculer le pourcentage de valeurs manquantes pour les colonnes numériques
                missing_percentage = numeric_cols.isna().mean()
                # Étape 4 : Supprimer les colonnes numériques avec trop de NaN
                numeric_cols_cleaned = numeric_cols.loc[:, missing_percentage < threshold]
                # Étape 5 : Remplir les valeurs manquantes dans les colonnes numériques restantes avec la médiane
                numeric_cols_cleaned = numeric_cols_cleaned.apply(lambda col: col.fillna(col.median()), axis=0)
                # Étape 6 : Réassembler : conserver les colonnes non numériques inchangées
                df_cleaned = pd.concat([non_numeric_cols, numeric_cols_cleaned], axis=1)
                cleaned_data[participant][activity][modality] = df_cleaned

    return cleaned_data

# cleaning values in modalities
def clean_missing_values_in_modalities(processed_data, threshold=0.5):

    zero_threshold = (processed_data == 0).mean(axis=1)  # Calcul du pourcentage de zéros par ligne
    rows_to_delete = zero_threshold[zero_threshold >= threshold]
    print(f"Lignes supprimées (trop de zéros):\n{rows_to_delete}")
    processed_data = processed_data.loc[zero_threshold < threshold]
    print(f"Nombre de lignes avant suppression: {len(zero_threshold)}")
    print(f"Nombre de lignes après suppression: {len(processed_data)}")
    numeric_cols = processed_data.select_dtypes(include=[np.number])
    non_numeric_cols = processed_data.select_dtypes(exclude=[np.number])
    missing_percentage = numeric_cols.isna().mean()
    numeric_cols_cleaned = numeric_cols.loc[:, missing_percentage < threshold]
    numeric_cols_cleaned = numeric_cols_cleaned.apply(lambda col: col.fillna(col.median()), axis=0)
    processed_data_cleaned = pd.concat([non_numeric_cols, numeric_cols_cleaned], axis=1)

    return processed_data_cleaned

In [ ]:
##  Normalisation des données
def normalize_data(cleaned_data):
    normalized_data = {}
    scaler = StandardScaler()

    for participant, activities in cleaned_data.items():
        normalized_data[participant] = {}
        for activity, modalities in activities.items():
            normalized_data[participant][activity] = {}
            for modality, df in modalities.items():
                # Vérifier si la première ligne contient des chaînes de caractères (en-têtes)
                if isinstance(df.iloc[0, 0], str):
                    headers = df.columns
                    df_values = df.iloc[1:].astype(float)
                    # Conserver les en-têtes en réappliquant après normalisation
                    df_normalized_values = pd.DataFrame(scaler.fit_transform(df_values), columns=headers, index=df.index[1:])
                    df_normalized = pd.concat([df.iloc[:1], df_normalized_values])
                else:
                    df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)

                normalized_data[participant][activity][modality] = df_normalized

    return normalized_data
# normalize in modality
def normalize_data_in_modalities(processed_data, columns_to_ignore=['id', 'categorie']):
    """
    Applique une normalisation Standard (StandardScaler) sur les données numériques,
    en ignorant certaines colonnes (comme 'id' et 'categorie').
    """
    # Exclure les colonnes à ignorer de la normalisation
    columns_to_normalize = processed_data.drop(columns=columns_to_ignore, errors='ignore')
    # Sélectionner uniquement les colonnes numériques restantes
    numeric_cols = columns_to_normalize.select_dtypes(include=[np.number])
    # Initialiser le StandardScaler
    scaler = StandardScaler()
    # Appliquer la normalisation sur les colonnes numériques
    numeric_cols_scaled = scaler.fit_transform(numeric_cols)
    # Convertir le résultat en DataFrame et garder les mêmes noms de colonnes
    numeric_cols_scaled_df = pd.DataFrame(numeric_cols_scaled, columns=numeric_cols.columns)
    # Réassembler les données après normalisation avec les colonnes non numériques intactes
    non_numeric_cols = processed_data[columns_to_ignore]
    processed_data_normalized = pd.concat([non_numeric_cols, numeric_cols_scaled_df], axis=1)

    return processed_data_normalized

In [ ]:
## Segmentation des données en fenêtres temporelles
def segment_data(df, window_size=100, step_size=50):
    segments = []
    num_frames = df.shape[0]

    for start in range(0, num_frames - window_size + 1, step_size):
        end = start + window_size
        segment = df.iloc[start:end].values  # Segmenter le dataframe
        segments.append(segment)

    return np.array(segments)

def segment_modalities(processed_data, window_size=100, step_size=50):
    segmented_data = {}

    for participant, activities in processed_data.items():
        segmented_data[participant] = {}
        for activity, modalities in activities.items():
            segmented_data[participant][activity] = {}

            for modality, df in modalities.items():
                segmented_data[participant][activity][modality] = segment_data(df, window_size, step_size)

    return segmented_data


In [ ]:
newFile = pd.read_csv('/content/drive/MyDrive/MovePort/19/forward/ips_1.csv')
newFile

,ips_1,emg_1,imu_1
0,0.933233,1.085426,-0.189408
1,0.348409,0.048023,-0.299162
2,2.691943,-0.091726,-0.901430
3,-2.073614,-1.135813,-0.297588
4,-0.831253,0.383549,2.082323
...,...,...,...
95,-0.462653,-0.561040,-0.977154
96,-0.184851,0.010057,0.880010
97,0.545298,0.124867,0.917960
98,0.017803,0.617142,-0.569876


In [ ]:
def convert_object_to_numeric(data):
    for col in data.columns:
        if data[col].dtype == 'object':  # Vérifie si la colonne est de type 'object'
            # Tente de convertir les valeurs en numériques
            data[col] = pd.to_numeric(data[col], errors='coerce')  # 'coerce' transforme les erreurs en NaN
    return data

In [ ]:
modalityImu = 'imu'  # Modalité, ici IMU
modalityEmg = 'emg'  # Modalité, ici EMG
modalityIps = 'ips'  # Modalité, ici IPS

# Appel de la fonction pour traiter les données
processed_dataImu = process_participants(data_path, participants, modalityImu, batch_size)
processed_dataEmg = process_participants(data_path, participants, modalityEmg, batch_size)
processed_dataIps = process_participants(data_path, participants, modalityIps, batch_size)

processed_dataEmg.head(1000)

In [ ]:
# Liste des participants à traiter
participants_to_process = [1, 2]

In [ ]:
# 1 : Charger et prétraiter les données
processed_data = process_data_in_batches(data_path, participants_to_process, batch_size=5)


Traitement du lot 1 : [1, 2]
Participant: 1
  Activity: forward
  Activity: still
  Activity: back
  Activity: halfsquat
  Données traitées pour le participant 1:
{'forward': {'EMG':                   0         1          2         3         4         5   \
Unnamed: 0    R_Vlat      R_RF       R_ST      R_TA    L_Vlat      L_RF   
0          -4.431152  3.323364   7.653809 -4.733276  3.927612  2.920532   
1          -9.970093  0.100708  31.723022 -4.733276   0.50354  3.726196   
2          -6.546021  2.920532  40.484619 -4.733276  2.215576 -0.704956   
3          -1.208496  0.302124  47.232056 -4.733276 -4.632568  0.402832   
...              ...       ...        ...       ...       ...       ...   
120155     -7.351685 -3.826905 -61.029053 -1.611328   2.01416 -1.309204   
120156     -7.452393  4.833984 -55.892944   1.00708 -4.733276 -2.920532   
120157     -4.934692  3.121948   -56.5979  0.402832  4.330444 -2.719116   
120158      -3.52478 -0.201416 -68.582153  0.604248  -3.52478 -3.22

In [ ]:
#print("head of IPS data:", processed_data[1]['forward']['IPS'].head(2))
#processed_data[1]['forward']['IPS'].head(2)
#print("head of EMG data:", processed_data[1]['forward']['EMG'].head(2))
#print("head of IMU data:", processed_data[1]['forward']['IMU'].head(2))

In [ ]:
# Étape 2 : Rééchantillonner les données pour aligner les fréquences
#resampled_data = resample_modalities(processed_data, target_rate=100)
type(processed_data)
# Si 'forward' contient plusieurs modalités (EMG, IMU, IPS)
print('################### FORWARD')
print("Shape of IPS data:", processed_data[1]['forward']['IPS'].shape)
print("Shape of EMG data:", processed_data[1]['forward']['EMG'].shape)
print("Shape of IMU data:", processed_data[1]['forward']['IMU'].shape)
print('################### BACK')
print("Shape of IPS data:", processed_data[1]['back']['IPS'].shape)
print("Shape of EMG data:", processed_data[1]['back']['EMG'].shape)
print("Shape of IMU data:", processed_data[1]['back']['IMU'].shape)
print('################### HALFSQUAT')
print("Shape of IPS data:", processed_data[1]['halfsquat']['IPS'].shape)
print("Shape of EMG data:", processed_data[1]['halfsquat']['EMG'].shape)
print("Shape of IMU data:", processed_data[1]['halfsquat']['IMU'].shape)
print('################### STILL')
print("Shape of IPS data:", processed_data[1]['still']['IPS'].shape)
print("Shape of EMG data:", processed_data[1]['still']['EMG'].shape)
print("Shape of IMU data:", processed_data[1]['still']['IMU'].shape)


################### FORWARD
Shape of IPS data: (3605, 682)
Shape of EMG data: (120161, 16)
Shape of IMU data: (6009, 54)
################### BACK
Shape of IPS data: (3974, 682)
Shape of EMG data: (132441, 16)
Shape of IMU data: (6623, 54)
################### HALFSQUAT
Shape of IPS data: (3583, 682)
Shape of EMG data: (119401, 16)
Shape of IMU data: (5971, 54)
################### STILL
Shape of IPS data: (3949, 682)
Shape of EMG data: (131621, 16)
Shape of IMU data: (6582, 54)


In [ ]:
# Étape 3 : Nettoyer les valeurs manquantes
# Nettoyage des données avec un seuil de 50% pour supprimer les colonnes trop vides
cleaned_data = clean_missing_values(processed_data, threshold=0.51)
# Si 'forward' contient plusieurs modalités (EMG, IMU, IPS)

In [ ]:
print('################### FORWARD')
print("Shape of IPS data:", cleaned_data[1]['forward']['IPS'].shape)
print("Shape of EMG data:", cleaned_data[1]['forward']['EMG'].shape)
print("Shape of IMU data:", cleaned_data[1]['forward']['IMU'].shape)
print('################### BACK')
print("Shape of IPS data:", cleaned_data[1]['back']['IPS'].shape)
print("Shape of EMG data:", cleaned_data[1]['back']['EMG'].shape)
print("Shape of IMU data:", cleaned_data[1]['back']['IMU'].shape)
print('################### HALFSQUAT')
print("Shape of IPS data:", cleaned_data[1]['halfsquat']['IPS'].shape)
print("Shape of EMG data:", cleaned_data[1]['halfsquat']['EMG'].shape)
print("Shape of IMU data:", cleaned_data[1]['halfsquat']['IMU'].shape)
print('################### STILL')
print("Shape of IPS data:", cleaned_data[1]['still']['IPS'].shape)
print("Shape of EMG data:", cleaned_data[1]['still']['EMG'].shape)
print("Shape of IMU data:", cleaned_data[1]['still']['IMU'].shape)

################### FORWARD
Shape of IPS data: (3605, 682)
Shape of EMG data: (120161, 16)
Shape of IMU data: (6007, 54)
################### BACK
Shape of IPS data: (1232, 682)
Shape of EMG data: (132441, 16)
Shape of IMU data: (6623, 54)
################### HALFSQUAT
Shape of IPS data: (3553, 682)
Shape of EMG data: (119401, 16)
Shape of IMU data: (5969, 54)
################### STILL
Shape of IPS data: (3949, 682)
Shape of EMG data: (131621, 16)
Shape of IMU data: (6580, 54)


In [ ]:
cleaned_data[1]['forward']['EMG']

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
Unnamed: 0,R_Vlat,R_RF,R_ST,R_TA,L_Vlat,L_RF,L_ST,L_TA,R_MG,R_LG,R_SOL,R_IL,L_MG,L_LG,L_SOL,L_IL
0,-4.431152,3.323364,7.653809,-4.733276,3.927612,2.920532,29.205322,-3.625488,-1.913452,1.00708,0.704956,-5.438232,3.222656,-6.747437,-0.50354,3.323364
1,-9.970093,0.100708,31.723022,-4.733276,0.50354,3.726196,34.844971,-5.841064,-9.869385,3.323364,7.754517,-5.438232,4.129028,2.5177,-0.50354,3.323364
2,-6.546021,2.920532,40.484619,-4.733276,2.215576,-0.704956,38.571167,-0.906372,-3.927612,-5.639648,1.00708,-5.438232,2.316284,-3.625488,-0.50354,3.323364
3,-1.208496,0.302124,47.232056,-4.733276,-4.632568,0.402832,44.512939,3.424072,-0.906372,-2.920532,6.646729,-5.438232,0.302124,0.604248,-0.50354,3.323364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120155,-7.351685,-3.826905,-61.029053,-1.611328,2.01416,-1.309204,20.947266,-3.121948,-12.185669,10.775757,20.141602,3.222656,4.53186,6.344604,0.704956,-6.04248
120156,-7.452393,4.833984,-55.892944,1.00708,-4.733276,-2.920532,17.422485,-8.258057,-13.092041,45.721436,15.811158,6.143188,0.402832,7.955932,-4.02832,-5.941772
120157,-4.934692,3.121948,-56.5979,0.402832,4.330444,-2.719116,21.350098,-6.848145,-29.306028,42.398071,13.092041,3.927612,-0.302124,7.855225,-2.01416,-12.689209
120158,-3.52478,-0.201416,-68.582153,0.604248,-3.52478,-3.222656,35.952759,-6.848145,-35.952759,20.040894,5.53894,2.215576,-0.604248,7.855225,-5.740356,-13.595581


In [ ]:
def extract_features_with_segments(emg_data, window_size=2000, sampling_rate=2000):
    """
    Extrait des caractéristiques pour chaque segment d'un signal EMG.
    - window_size : Nombre d'échantillons par segment (ex. : 2000 = 1 seconde).
    """
    # Si la première ligne contient des en-têtes, l'ignorer
    if not np.issubdtype(emg_data.iloc[0].dtype, np.number):
        emg_data = emg_data.iloc[1:]  # Ignorer la première ligne
    num_segments = emg_data.shape[0] // window_size
    features_list = []

    for i in range(num_segments):
        segment = emg_data.iloc[i * window_size:(i + 1) * window_size]

        # Calculer les caractéristiques pour ce segment
        segment_features = {}
        for muscle in segment.columns:
            signal = segment[muscle]
            # RMS
            rms = np.sqrt(np.mean(signal ** 2))
            # Mean Absolute Value
            mav = np.mean(np.abs(signal))
            # Median Frequency
            freqs = np.fft.fftfreq(len(signal), d=1 / sampling_rate)
            spectrum = np.abs(np.fft.fft(signal))
            median_freq = np.median(freqs[np.argmax(spectrum)])

            # Ajouter les caractéristiques au dictionnaire
            segment_features[muscle] = {"RMS": rms, "MAV": mav, "MedianFreq": median_freq}

        # Convertir les caractéristiques en DataFrame
        features_df = pd.DataFrame(segment_features).T
        features_list.append(features_df)

    # Concaténer toutes les caractéristiques
    return pd.concat(features_list, ignore_index=True)

# Exemple d'utilisation :
emg_data = cleaned_data[1]['forward']['EMG']  # Vos données EMG
segmented_features = extract_features_with_segments(emg_data, window_size=2000)
print(segmented_features)


           RMS        MAV  MedianFreq
0     5.034728   3.980562         0.0
1     4.455879   3.582912         0.0
2    57.840199  43.563112        60.0
3     8.357357   6.085133         0.0
4     4.548004   3.598988         0.0
..         ...        ...         ...
955   4.514225   3.527198         0.0
956   4.666401   3.568422         1.0
957   3.594585   2.868886         0.0
958   4.161047   3.308334         0.0
959   4.773785   3.706633         0.0

[960 rows x 3 columns]


In [ ]:
def extract_emg_features(emg_data, sampling_rate=2000):
  # Si la première ligne contient des en-têtes, l'ignorer
    if not np.issubdtype(emg_data.iloc[0].dtype, np.number):
        emg_data = emg_data.iloc[1:]  # Ignorer la première ligne
    features = {}
    # Calcul des caractéristiques pour chaque colonne (chaque muscle)
    for muscle in emg_data.columns:
        signal = emg_data[muscle]
        # RMS
        rms = np.sqrt(np.mean(signal ** 2))
        # Mean Absolute Value
        mav = np.mean(np.abs(signal))
        # Median Frequency
        freqs = np.fft.fftfreq(len(signal), d=1 / sampling_rate)
        spectrum = np.abs(np.fft.fft(signal))
        median_freq = np.median(freqs[np.argmax(spectrum)])
        features[muscle] = {"RMS": rms, "MAV": mav, "MedianFreq": median_freq}
    # Convertir en DataFrame pour une meilleure manipulation
    features_df = pd.DataFrame(features).T
    return features_df
def extract_features_with_segments(emg_data, window_size=2000, sampling_rate=2000):
    """
    Extrait des caractéristiques pour chaque segment d'un signal EMG.
    - window_size : Nombre d'échantillons par segment (ex. : 2000 = 1 seconde).
    """
    num_segments = emg_data.shape[0] // window_size
    features_list = []

    for i in range(num_segments):
        segment = emg_data.iloc[i * window_size:(i + 1) * window_size]
        features = extract_emg_features(segment, sampling_rate)
        features_list.append(features)

    # Concaténer toutes les caractéristiques
    return pd.concat(features_list, ignore_index=True)

# Exemple d'utilisation :
emg_data = cleaned_data[1]['forward']['EMG']
segmented_features = extract_features_with_segments(emg_data, window_size=2000)
#print(segmented_features)

#emg_data = pd.DataFrame(np.random.randn(2000, 16), columns=[f"Muscle_{i+1}" for i in range(16)])  # Signal synthétique
#emg_data = cleaned_data[1]['forward']['EMG']
#emg_features = extract_emg_features(emg_data)



In [ ]:
#print(emg_features)
print("RMS", segmented_features["RMS"])
print("      MAV", segmented_features["MAV"])
print("MedianFreq", segmented_features["MedianFreq"])
print("      Dimensions des caractéristiques EMG :", segmented_features.shape)
print("      caractéristiques EMG :", segmented_features)

RMS 0       5.035824
1       4.453617
2      57.847613
3       8.359447
4       4.549061
         ...    
955     4.515354
956     4.667255
957     3.592409
958     4.158927
959     4.774966
Name: RMS, Length: 960, dtype: float64
      MAV 0       3.981647
1       3.580826
2      43.564702
3       6.088127
4       3.600184
         ...    
955     3.528912
956     3.568998
957     2.866996
958     3.306362
959     3.708236
Name: MAV, Length: 960, dtype: float64
MedianFreq 0       0.000000
1       0.000000
2      60.030015
3       0.000000
4       0.000000
         ...    
955     0.000000
956     1.000500
957     0.000000
958     0.000000
959     0.000000
Name: MedianFreq, Length: 960, dtype: float64
      Dimensions des caractéristiques EMG : (960, 3)
      caractéristiques EMG :            RMS        MAV  MedianFreq
0     5.035824   3.981647    0.000000
1     4.453617   3.580826    0.000000
2    57.847613  43.564702   60.030015
3     8.359447   6.088127    0.000000
4     4.549061   3

In [ ]:
# Étape 4 : Normaliser les données
normalized_data = normalize_data(cleaned_data)

In [ ]:
# Participant à vérifier
participant_id = 1

# Parcourir les activités et modalités pour ce participant
for activity, modalities in normalized_data[participant_id].items():
    print(f"Activité : {activity}")

    for modality, df in modalities.items():
        print(f"  Modalité : {modality}, Taille des données : {df.shape}")

        # Afficher un aperçu des 5 premières lignes de la modalité
        print(f"    Aperçu des 5 premières lignes de {modality} :")
        print(df.head())  # Afficher les premières lignes du DataFrame
        print("-" * 50)  # Ligne de séparation pour rendre la sortie plus lisible

Activité : forward
  Modalité : EMG, Taille des données : (120161, 16)
    Aperçu des 5 premières lignes de EMG :
                  0         1         2         3         4         5   \
Unnamed: 0    R_Vlat      R_RF      R_ST      R_TA    L_Vlat      L_RF   
0          -0.699717  1.588466  0.228831 -0.174003  1.204777   1.25631   
1          -1.901176  0.701886  0.750127 -0.174003  0.406394  1.460689   
2          -1.158456  1.477643  0.939888 -0.174003  0.805585  0.336604   
3          -0.000686  0.757297  1.086025 -0.174003 -0.791181  0.617625   

                  6         7         8         9         10        11  \
Unnamed: 0      L_ST      L_TA      R_MG      R_LG     R_SOL      R_IL   
0           1.058997 -0.015547  0.031426  0.086216  0.101264 -1.216718   
1           1.249201 -0.155308 -0.647623  0.453086  0.540769 -1.216718   
2           1.374871  0.155979 -0.140485 -0.966541  0.120099 -1.216718   
3           1.575264  0.429149  0.117381 -0.535868  0.471704 -1.216718 

In [ ]:
# Étape 5 : Segmenter les données en fenêtres temporelles
segmented_data = segment_modalities(normalized_data, window_size=100, step_size=50)

In [ ]:
# Étape 6 : Reshaper les données IPS en matrices
reshaped_data = segment_and_reshape_modalities(segmented_data)

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Pour les analyses statistiques
from scipy.signal import resample
from scipy.stats import describe

In [ ]:
"""import os
import pandas as pd
import glob

def inverser_csv(file_path):
    """
    Lire un fichier CSV et le transposer.
    """
    df = pd.read_csv(file_path, header=0)
    df = df.transpose()
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)
    return df

def process_participants_by_modality(data_path, participants, categories, modality, output_file="processed_data.csv"):
    """
    Traite les fichiers de plusieurs participants pour une modalité spécifique (EMG, IMU ou IPS).
    Ajoute des colonnes 'id' et 'categorie' et renvoie les données concaténées.
    """
    all_data = []  # Pour stocker toutes les données concaténées

    # Parcourir les participants
    for participant_id in participants:
        participant_path = os.path.join(data_path, str(participant_id))

        for category in categories:
            category_path = os.path.join(participant_path, category)

            # Chercher les fichiers spécifiques à la modalité
            files = glob.glob(os.path.join(category_path, f"{modality}_*.csv"))

            print(f"Participant {participant_id}, Catégorie : {category}")
            print(f"  Fichiers {modality.upper()} : {files}")

            category_data = []  # Liste temporaire pour stocker les données de la catégorie actuelle

            # Traiter chaque fichier
            for file in files:
                print(f"    Traitement du fichier {modality.upper()} : {file}")
                try:
                    df = inverser_csv(file)  # Utiliser la fonction d'inversion
                    df['id'] = participant_id  # Ajouter l'ID du participant
                    df['categorie'] = category  # Ajouter la catégorie
                    category_data.append(df)  # Ajouter au lot de la catégorie
                except Exception as e:
                    print(f"Erreur lors du traitement du fichier {file}: {e}")

            # Ajouter les données de la catégorie au lot global
            if category_data:
                concatenated_category_data = pd.concat(category_data, axis=0, ignore_index=True)
                all_data.append(concatenated_category_data)

    # Concaténer toutes les données en un seul DataFrame
    if all_data:
        concatenated_data = pd.concat(all_data, axis=0, ignore_index=True)

        # Sauvegarder dans un fichier CSV
        concatenated_data.to_csv(output_file, index=False)
        print(f"Les données ont été sauvegardées dans le fichier {output_file}.")

        return concatenated_data
    else:
        print("Aucune donnée valide trouvée.")
        return None

# Configuration
data_path = '/content/drive/MyDrive/MovePort'
participants = list(range(1, 26))  # Participants 1 et 2
categories = ['back', 'forward', 'halfsquat', 'still']
modality = "emg"  # Spécifiez la modalité : "emg", "imu", ou "ips"
output_file = "processed_data_Emg.csv"  # Nom du fichier de sortie

# Exécution
all_data = process_participants_by_modality(data_path, participants, categories, modality, output_file)

# Afficher les premières lignes (head) de toutes les données traitées
if all_data is not None:
    print("Aperçu des premières lignes des données traitées :")
    print(all_data.head())"""

In [ ]:
def process_participants_by_modality(data_path, participants, categories, modality, batch_size=5):
    """
    Traite les fichiers de plusieurs participants pour une modalité spécifique (EMG, IMU ou IPS).
    Ajoute des colonnes 'id' et 'categorie' et enregistre les données par lots.
    """
    all_batches_files = []  # Pour stocker les chemins des fichiers batch
    participants_batches = [participants[i:i + batch_size] for i in range(0, len(participants), batch_size)]

    for batch_idx, batch_participants in enumerate(participants_batches, start=1):
        print(f"Traitement du lot {batch_idx}/{len(participants_batches)} : Participants {batch_participants}")
        batch_data = []  # Stocker les DataFrames d'un lot

        for participant_id in batch_participants:
            participant_path = os.path.join(data_path, str(participant_id))

            for category in categories:
                category_path = os.path.join(participant_path, category)

                # Chercher les fichiers spécifiques à la modalité
                files = glob.glob(os.path.join(category_path, f"{modality}_*.csv"))

                print(f"Participant {participant_id}, Catégorie : {category}")
                print(f"  Fichiers {modality.upper()} : {files}")

                category_data = []  # Liste temporaire pour stocker les données de la catégorie actuelle

                for file in files:
                    print(f"    Traitement du fichier {modality.upper()} : {file}")
                    try:
                        df = inverser_csv(file)  # Utiliser la fonction d'inversion
                        df['id'] = participant_id  # Ajouter l'ID du participant
                        df['categorie'] = category  # Ajouter la catégorie
                        category_data.append(df)  # Ajouter au lot de la catégorie
                    except Exception as e:
                        print(f"Erreur lors du traitement du fichier {file}: {e}")

                # Ajouter les données de la catégorie au lot global
                if category_data:
                    concatenated_category_data = pd.concat(category_data, axis=0, ignore_index=True)
                    batch_data.append(concatenated_category_data)

        # Concaténer les données du lot et sauvegarder
        if batch_data:
            batch_df = pd.concat(batch_data, ignore_index=True)
            batch_file = f"{modality}_batch_{batch_idx}.csv"
            batch_df.to_csv(batch_file, index=False)
            all_batches_files.append(batch_file)
            print(f"Fichier batch {batch_file} enregistré.")
        else:
            print(f"Aucune donnée valide trouvée pour le lot {batch_idx}.")

    return all_batches_files

# Configuration
data_path = '/content/drive/MyDrive/MovePort'
participants = list(range(1, 6))  # Participants 1 et 2
categories = ['back', 'forward', 'halfsquat', 'still']
modality = "emg"  # Spécifiez la modalité : "emg", "imu", ou "ips"
batch_size = 5

# Exécution
batch_files = process_participants_by_modality(data_path, participants, categories, modality, batch_size)
print(f"Fichiers batch générés pour la modalité {modality.upper()} : {batch_files}")

In [ ]:
"""1 -> back -> ips_1.csv , emg_1.csv, imu_1.csv
1 -> forward -> ips_1.csv , emg_1.csv, imu_1.csv
1 -> halfsquat -> ips_1.csv , emg_1.csv, imu_1.csv
1 -> still -> ips_1.csv , emg_1.csv, imu_1.csv
2 -> back -> ips_1.csv , emg_1.csv, imu_1.csv
2 -> forward -> ips_1.csv , emg_1.csv, imu_1.csv
2 -> halfsquat -> ips_1.csv , emg_1.csv, imu_1.csv
2 -> still -> ips_1.csv , emg_1.csv, imu_1.csv
3 -> back -> ips_1.csv , emg_1.csv, imu_1.csv
3 -> forward -> ips_1.csv , emg_1.csv, imu_1.csv
3 -> halfsquat -> ips_1.csv , emg_1.csv, imu_1.csv
3 -> still -> ips_1.csv , emg_1.csv, imu_1.csv
...
25 -> back -> ips_1.csv , emg_1.csv, imu_1.csv
25 -> forward -> ips_1.csv , emg_1.csv, imu_1.csv
25 -> halfsquat -> ips_1.csv , emg_1.csv, imu_1.csv
25 -> still -> ips_1.csv , emg_1.csv, imu_1.csv"""


'1 -> back -> ips_1.csv , emg_1.csv, imu_1.csv\n1 -> forward -> ips_1.csv , emg_1.csv, imu_1.csv\n1 -> halfsquat -> ips_1.csv , emg_1.csv, imu_1.csv\n1 -> still -> ips_1.csv , emg_1.csv, imu_1.csv\n2 -> back -> ips_1.csv , emg_1.csv, imu_1.csv\n2 -> forward -> ips_1.csv , emg_1.csv, imu_1.csv\n2 -> halfsquat -> ips_1.csv , emg_1.csv, imu_1.csv\n2 -> still -> ips_1.csv , emg_1.csv, imu_1.csv\n3 -> back -> ips_1.csv , emg_1.csv, imu_1.csv\n3 -> forward -> ips_1.csv , emg_1.csv, imu_1.csv\n3 -> halfsquat -> ips_1.csv , emg_1.csv, imu_1.csv\n3 -> still -> ips_1.csv , emg_1.csv, imu_1.csv\n...\n25 -> back -> ips_1.csv , emg_1.csv, imu_1.csv\n25 -> forward -> ips_1.csv , emg_1.csv, imu_1.csv\n25 -> halfsquat -> ips_1.csv , emg_1.csv, imu_1.csv\n25 -> still -> ips_1.csv , emg_1.csv, imu_1.csv'